### Import Libraries and Pandas Setting
The setting is changed to ignore the chained assignment warning. If users plan to keep the original dataframe, consider using .copy() to avoid the warning manually.

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

### Crude Oil Price from EIA
This was written before the Crude Oil Price data was provided by Henkel. The data was taken from the U.S. Energy Information Administration and submitted along with rest of the data. However, in the final model, only data from Henkel is used.

In [2]:
#EIA
df_crudeoilprice = pd.read_excel('CrudeOil_Price.xls', sheet_name = 'Data 1', header = 2)
df_crudeoilprice['Year'] = df_crudeoilprice['Date'].dt.year
df_crudeoilprice['Month'] = df_crudeoilprice['Date'].dt.month
df_crudeoilprice = df_crudeoilprice[(df_crudeoilprice['Year'] >= 2015) & (df_crudeoilprice['Year'] <= 2023)]
df_crudeoilprice.drop(columns = ['Date'], inplace = True)
df_crudeoilprice.rename(columns={"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)": "CrudeOilPrice_WTI",
                                 "Europe Brent Spot Price FOB (Dollars per Barrel)": "CrudeOilPrice_Brent"}, inplace = True)
df_crudeoilprice = df_crudeoilprice.groupby(['Year', 'Month']).mean()
df_crudeoilprice.reset_index(inplace = True)
display(df_crudeoilprice)

,Year,Month,CrudeOilPrice_WTI,CrudeOilPrice_Brent
0,2015,1,47.219000,47.759500
1,2015,2,50.584211,58.095500
2,2015,3,47.823636,55.885455
3,2015,4,54.452857,59.524286
4,2015,5,59.265000,64.075000
...,...,...,...,...
103,2023,8,81.386087,86.147727
104,2023,9,89.425000,93.722381
105,2023,10,85.639524,90.595909
106,2023,11,77.685000,82.940000


### Crude Oil Price from Henkel
The specific format of the date can be specified to avoid the message. However, this warning does not affect the functionality of the script.

In [3]:
df_crudeoilprice = pd.read_excel('Henkel_Crude&DieselPricesEurope.xlsx',
                                 header = 9)
df_crudeoilprice = df_crudeoilprice.iloc[:, 1:4]
df_crudeoilprice.rename(columns = {'Unnamed: 1': 'Date',
                                   'Unnamed: 2': 'CrudeOilPrice_EU_Brent',
                                   'Unnamed: 3': 'CrudeOilPrice_US_WTI'},
                        inplace = True)
df_crudeoilprice['Date'] = pd.to_datetime(df_crudeoilprice['Date'], errors='coerce')

#print(df_crudeoilprice.dtypes)
df_crudeoilprice['Year'] = df_crudeoilprice['Date'].dt.year
df_crudeoilprice['Month'] = df_crudeoilprice['Date'].dt.month
df_crudeoilprice = df_crudeoilprice[(df_crudeoilprice['Year'] >= 2015) & (df_crudeoilprice['Year'] <= 2023)]
df_crudeoilprice = df_crudeoilprice[['Year', 'Month', 'CrudeOilPrice_EU_Brent', 'CrudeOilPrice_US_WTI']]
display(df_crudeoilprice)

C:\Users\Kydo\AppData\Local\Temp\ipykernel_7436\3778952484.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_crudeoilprice['Date'] = pd.to_datetime(df_crudeoilprice['Date'], errors='coerce')


,Year,Month,CrudeOilPrice_EU_Brent,CrudeOilPrice_US_WTI
84,2015,1,47.5,47.3
85,2015,2,58.2,50.6
86,2015,3,56.4,47.8
87,2015,4,59.5,54.5
88,2015,5,64.0,59.3
...,...,...,...,...
187,2023,8,86.2,81.4
188,2023,9,94.0,89.6
189,2023,10,91.1,85.6
190,2023,11,83.1,77.4


### Exchange Rate from Investing.com

In [4]:
#Investing
df_eurusd = pd.read_csv('EUR_USD.csv')
df_eurusd['Date'] = pd.to_datetime(df_eurusd['Date'], format='%m/%d/%Y')
df_eurusd['Year'] = df_eurusd['Date'].dt.year
df_eurusd['Month'] = df_eurusd['Date'].dt.month
df_eurusd = df_eurusd[df_eurusd['Year'] <= 2023]
df_eurusd = df_eurusd[['Year', 'Month', 'Price']]
df_eurusd.rename(columns={"Price": "EUR_USD_Rate"}, inplace = True)
display(df_eurusd)

,Year,Month,EUR_USD_Rate
1,2023,12,1.1037
2,2023,11,1.0886
3,2023,10,1.0576
4,2023,9,1.0570
5,2023,8,1.0841
...,...,...,...
104,2015,5,1.0987
105,2015,4,1.1222
106,2015,3,1.0730
107,2015,2,1.1193


### GDP, GDP Change, and Inflation from the IMF

In [5]:
#IMF
temp = pd.read_excel('GDP.xlsx')
temp.drop(columns = ['Scale', 'Country/Series-specific Notes'], inplace = True)
##
df_GDP = temp[temp['Subject Descriptor'] == 'Gross domestic product, current prices']
df_GDP.drop(columns = ['Subject Descriptor', 'Units'], inplace = True)
df_GDP.set_index('Country Group Name', inplace=True)
df_GDP = df_GDP.transpose()
df_GDP.reset_index(inplace = True)
df_GDP = df_GDP.rename_axis(None, axis=1)
df_GDP.columns = df_GDP.columns.map(lambda x: 'GDP_' + str(x))
df_GDP.rename(columns = {'GDP_index': 'Year'}, inplace = True)
df_GDP['Month'] = [range(1, 13)]*9
df_GDP = df_GDP.explode('Month', ignore_index=True)
display(df_GDP)

,Year,GDP_World,GDP_Advanced economies,GDP_Euro area,GDP_Major advanced economies (G7),GDP_Other advanced economies (Advanced economies excluding G7 and euro area),GDP_European Union,GDP_ASEAN-5,GDP_Emerging market and developing economies,GDP_Emerging and developing Asia,GDP_Emerging and developing Europe,GDP_Latin America and the Caribbean,GDP_Middle East and Central Asia,GDP_Sub-Saharan Africa,Month
0,2015,75011.41,45448.99,11727.50,34777.16,6577.99,13554.45,2177.84,29562.43,15825.04,3294.24,5132.41,3682.75,1627.99,1
1,2015,75011.41,45448.99,11727.50,34777.16,6577.99,13554.45,2177.84,29562.43,15825.04,3294.24,5132.41,3682.75,1627.99,2
2,2015,75011.41,45448.99,11727.50,34777.16,6577.99,13554.45,2177.84,29562.43,15825.04,3294.24,5132.41,3682.75,1627.99,3
3,2015,75011.41,45448.99,11727.50,34777.16,6577.99,13554.45,2177.84,29562.43,15825.04,3294.24,5132.41,3682.75,1627.99,4
4,2015,75011.41,45448.99,11727.50,34777.16,6577.99,13554.45,2177.84,29562.43,15825.04,3294.24,5132.41,3682.75,1627.99,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023,104476.43,60929.56,15478.35,46295.31,8800.73,18351.13,3293.50,43546.87,25441.04,4933.18,6517.25,4698.17,1957.23,8
104,2023,104476.43,60929.56,15478.35,46295.31,8800.73,18351.13,3293.50,43546.87,25441.04,4933.18,6517.25,4698.17,1957.23,9
105,2023,104476.43,60929.56,15478.35,46295.31,8800.73,18351.13,3293.50,43546.87,25441.04,4933.18,6517.25,4698.17,1957.23,10
106,2023,104476.43,60929.56,15478.35,46295.31,8800.73,18351.13,3293.50,43546.87,25441.04,4933.18,6517.25,4698.17,1957.23,11


In [6]:
#IMF
df_GDPchange = temp[temp['Subject Descriptor'] == 'Gross domestic product, constant prices']
df_GDPchange.drop(columns = ['Subject Descriptor', 'Units'], inplace = True)
df_GDPchange.set_index('Country Group Name', inplace=True)
df_GDPchange = df_GDPchange.transpose()
df_GDPchange.reset_index(inplace = True)
df_GDPchange = df_GDPchange.rename_axis(None, axis=1)
df_GDPchange.columns = df_GDPchange.columns.map(lambda x: 'GDPChange_' + str(x))
df_GDPchange.rename(columns = {'GDPChange_index': 'Year'}, inplace = True)
df_GDPchange['Month'] = [range(1, 13)]*9
df_GDPchange = df_GDPchange.explode('Month', ignore_index=True)
display(df_GDPchange)

,Year,GDPChange_World,GDPChange_Advanced economies,GDPChange_Euro area,GDPChange_Major advanced economies (G7),GDPChange_Other advanced economies (Advanced economies excluding G7 and euro area),GDPChange_European Union,GDPChange_ASEAN-5,GDPChange_Emerging market and developing economies,GDPChange_Emerging and developing Asia,GDPChange_Emerging and developing Europe,GDPChange_Latin America and the Caribbean,GDPChange_Middle East and Central Asia,GDPChange_Sub-Saharan Africa,Month
0,2015,3.427,2.292,2.026,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180,1
1,2015,3.427,2.292,2.026,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180,2
2,2015,3.427,2.292,2.026,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180,3
3,2015,3.427,2.292,2.026,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180,4
4,2015,3.427,2.292,2.026,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023,2.964,1.528,0.663,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346,8
104,2023,2.964,1.528,0.663,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346,9
105,2023,2.964,1.528,0.663,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346,10
106,2023,2.964,1.528,0.663,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346,11


In [7]:
#IMF
df_inflation = temp[temp['Subject Descriptor'] == 'Inflation, average consumer prices']
df_inflation.drop(columns = ['Subject Descriptor', 'Units'], inplace = True)
df_inflation.set_index('Country Group Name', inplace=True)
df_inflation = df_inflation.transpose()
df_inflation.reset_index(inplace = True)
df_inflation = df_inflation.rename_axis(None, axis=1)
df_inflation.columns = df_inflation.columns.map(lambda x: 'Inflation_' + str(x))
df_inflation.rename(columns = {'Inflation_index': 'Year'}, inplace = True)
df_inflation['Month'] = [range(1, 13)]*9
df_inflation = df_inflation.explode('Month', ignore_index=True)
display(df_inflation)

,Year,Inflation_World,Inflation_Advanced economies,Inflation_Euro area,Inflation_Major advanced economies (G7),Inflation_Other advanced economies (Advanced economies excluding G7 and euro area),Inflation_European Union,Inflation_ASEAN-5,Inflation_Emerging market and developing economies,Inflation_Emerging and developing Asia,Inflation_Emerging and developing Europe,Inflation_Latin America and the Caribbean,Inflation_Middle East and Central Asia,Inflation_Sub-Saharan Africa,Month
0,2015,2.748,0.307,0.193,0.311,0.565,0.102,3.105,4.770,2.721,10.749,5.406,5.577,6.718,1
1,2015,2.748,0.307,0.193,0.311,0.565,0.102,3.105,4.770,2.721,10.749,5.406,5.577,6.718,2
2,2015,2.748,0.307,0.193,0.311,0.565,0.102,3.105,4.770,2.721,10.749,5.406,5.577,6.718,3
3,2015,2.748,0.307,0.193,0.311,0.565,0.102,3.105,4.770,2.721,10.749,5.406,5.577,6.718,4
4,2015,2.748,0.307,0.193,0.311,0.565,0.102,3.105,4.770,2.721,10.749,5.406,5.577,6.718,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023,6.876,4.635,5.551,4.688,4.326,6.477,3.583,8.483,2.614,18.910,13.831,18.010,15.751,8
104,2023,6.876,4.635,5.551,4.688,4.326,6.477,3.583,8.483,2.614,18.910,13.831,18.010,15.751,9
105,2023,6.876,4.635,5.551,4.688,4.326,6.477,3.583,8.483,2.614,18.910,13.831,18.010,15.751,10
106,2023,6.876,4.635,5.551,4.688,4.326,6.477,3.583,8.483,2.614,18.910,13.831,18.010,15.751,11


### Gold price from the WCG (gold.org)

In [8]:
#Gold.org
df_gold = pd.read_excel('Gold.xlsx', sheet_name = 'Monthly_Avg', header = 5)
df_gold = df_gold[['Unnamed: 2', 'USD', 'EUR']]
df_gold.rename(columns = {'Unnamed: 2': 'Date', 'USD': 'Gold_USD', 'EUR': 'Gold_EUR'}, inplace = True)
df_gold['Year'] = df_gold['Date'].dt.year
df_gold['Month'] = df_gold['Date'].dt.month
df_gold = df_gold[['Year', 'Month', 'Gold_USD', 'Gold_EUR']]
df_gold = df_gold[(df_gold['Year'] >= 2015) & (df_gold['Year'] <= 2023)]
display(df_gold)

,Year,Month,Gold_USD,Gold_EUR
444,2015,1,1251.85,1079.20
445,2015,2,1227.19,1081.09
446,2015,3,1178.63,1089.54
447,2015,4,1197.91,1108.91
448,2015,5,1199.05,1073.71
...,...,...,...,...
547,2023,8,1920.03,1758.99
548,2023,9,1916.96,1795.58
549,2023,10,1913.04,1811.08
550,2023,11,1985.27,1835.58


### Stock Price from Investing.com
S&P500 is a representative for US stocks. STOXX600 is the equivalence of the EU standards.

In [9]:
#Investing
df_SP500 = pd.read_csv('S&P500.csv')
df_SP500['Date'] = pd.to_datetime(df_SP500['Date'], format='%m/%d/%Y')
df_SP500['Year'] = df_SP500['Date'].dt.year
df_SP500['Month'] = df_SP500['Date'].dt.month
df_SP500 = df_SP500[df_SP500['Year'] <= 2023]
df_SP500 = df_SP500[['Year', 'Month', 'Price']]
df_SP500.rename(columns={"Price": "SP500_Price"}, inplace = True)
#df_SP500['SP500_Price'] = df_SP500['SP500_Price'].astype('float64')
display(df_SP500)

,Year,Month,SP500_Price
0,2023,12,4769.83
1,2023,11,4567.78
2,2023,10,4193.80
3,2023,9,4288.05
4,2023,8,4507.66
...,...,...,...
103,2015,5,2107.39
104,2015,4,2085.51
105,2015,3,2067.89
106,2015,2,2104.50


In [10]:
#Investing
df_STOXX600 = pd.read_csv('STOXX600.csv')
df_STOXX600['Date'] = pd.to_datetime(df_STOXX600['Date'], format='%m/%d/%Y')
df_STOXX600['Year'] = df_STOXX600['Date'].dt.year
df_STOXX600['Month'] = df_STOXX600['Date'].dt.month
df_STOXX600 = df_STOXX600[df_STOXX600['Year'] <= 2023]
df_STOXX600 = df_STOXX600[['Year', 'Month', 'Price']]
df_STOXX600.rename(columns={"Price": "STOXX600_Price"}, inplace = True)
display(df_STOXX600)

,Year,Month,STOXX600_Price
1,2023,12,479.02
2,2023,11,461.61
3,2023,10,433.66
4,2023,9,450.22
5,2023,8,458.19
...,...,...,...
104,2015,5,399.87
105,2015,4,395.79
106,2015,3,397.30
107,2015,2,392.21


# EU/US Gasoline and Diesel Price from IEA

In [11]:
temp = pd.read_excel('Gas_Die.xlsx', sheet_name = "Table", header = 5)
temp.fillna(0, inplace = True)
temp = temp[((temp['PRODUCT'] == 'Gasoline (unit/litre)') | (temp['PRODUCT'] == 'Diesel (unit/litre)')) & (temp['UNIT'] == 'US dollars')]
EU_countries = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus",
                "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany",
                "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania",
                "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania",
                "Slovak Republic", "Slovenia", "Spain", "Sweden"]
#print(len(EU_countries))
df_EU = temp[ temp['COUNTRY'].isin(EU_countries) == True]
df_NA = temp[(temp['COUNTRY'] == 'United States') | (temp['COUNTRY'] == 'Canada')]
#display(temp)

In [12]:
df_EU_gas = df_EU[df_EU['PRODUCT'] == 'Gasoline (unit/litre)']
df_EU_gas.drop(columns = ['PRODUCT', 'FLOW', 'UNIT'], inplace = True)
#Dec 2020
#print(list(df_EU_gas.columns))


df_EU_gas.set_index('COUNTRY', inplace=True)
df_EU_gas = df_EU_gas.transpose()
df_EU_gas.reset_index(inplace = True)
df_EU_gas = df_EU_gas.rename_axis(None, axis=1)
df_EU_gas.rename(columns = {'index': 'Date'}, inplace = True)


fix_lst = ['Bulgaria', 'Czech Republic', 'Estonia', 'Greece', 'Poland', 'Slovenia', 'Spain']
for country in fix_lst:
    df_EU_gas[country] = df_EU_gas[country].interpolate(method='slinear').interpolate(method='linear')

#Convert object to numeric
cols = df_EU_gas.columns
df_EU_gas[cols[1:]] = df_EU_gas[cols[1:]].apply(pd.to_numeric, errors='coerce')    

#Calculate EU Average
df_EU_gas.set_index('Date', inplace=True)
df_EU_gas['Average'] = df_EU_gas.mean(axis=1)
df_EU_gas.reset_index(inplace=True)

df_EU_gas['Year'] = df_EU_gas['Date'].dt.year
df_EU_gas['Month'] = df_EU_gas['Date'].dt.month
df_EU_gas = df_EU_gas[['Year', 'Month', 'Average']]
df_EU_gas.rename(columns = {'Average': 'EU_Gasoline'}, inplace = True)
display(df_EU_gas)

C:\Users\Kydo\AppData\Local\Temp\ipykernel_7436\3140885534.py:16: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_EU_gas[country] = df_EU_gas[country].interpolate(method='slinear').interpolate(method='linear')


,Year,Month,EU_Gasoline
0,2015,1,1.417407
1,2015,2,1.407407
2,2015,3,1.418519
3,2015,4,1.446667
4,2015,5,1.529630
...,...,...,...
103,2023,8,1.870741
104,2023,9,1.862963
105,2023,10,1.795926
106,2023,11,1.777037


In [13]:
df_NA_gas = df_NA[df_NA['PRODUCT'] == 'Gasoline (unit/litre)']
df_NA_gas.drop(columns = ['PRODUCT', 'FLOW', 'UNIT'], inplace = True)
#Dec 2020
#print(list(df_NA_gas.columns))


df_NA_gas.set_index('COUNTRY', inplace=True)
df_NA_gas = df_NA_gas.transpose()
df_NA_gas.reset_index(inplace = True)
df_NA_gas = df_NA_gas.rename_axis(None, axis=1)
df_NA_gas.rename(columns = {'index': 'Date'}, inplace = True)

#Convert object to numeric
cols = df_NA_gas.columns
df_NA_gas[cols[1:]] = df_NA_gas[cols[1:]].apply(pd.to_numeric, errors='coerce')    

#Calculate EU Average
df_NA_gas.set_index('Date', inplace=True)
df_NA_gas['Average'] = df_NA_gas.mean(axis=1)
df_NA_gas.reset_index(inplace=True)

df_NA_gas['Year'] = df_NA_gas['Date'].dt.year
df_NA_gas['Month'] = df_NA_gas['Date'].dt.month
df_NA_gas = df_NA_gas[['Year', 'Month', 'Average']]
df_NA_gas.rename(columns = {'Average': 'NA_Gasoline'}, inplace = True)
display(df_NA_gas)

,Year,Month,NA_Gasoline
0,2015,1,0.665
1,2015,2,0.700
2,2015,3,0.755
3,2015,4,0.770
4,2015,5,0.830
...,...,...,...
103,2023,8,1.150
104,2023,9,1.140
105,2023,10,1.055
106,2023,11,1.000


In [14]:
df_EU_diesel = df_EU[df_EU['PRODUCT'] == 'Diesel (unit/litre)']
df_EU_diesel.drop(columns = ['PRODUCT', 'FLOW', 'UNIT'], inplace = True)
#Dec 2020
#print(list(df_EU_diesel.columns))


df_EU_diesel.set_index('COUNTRY', inplace=True)
df_EU_diesel = df_EU_diesel.transpose()
df_EU_diesel.reset_index(inplace = True)
df_EU_diesel = df_EU_diesel.rename_axis(None, axis=1)
df_EU_diesel.rename(columns = {'index': 'Date'}, inplace = True)


fix_lst = ['Bulgaria', 'Czech Republic', 'Estonia', 'Greece', 'Poland', 'Slovenia', 'Spain']
for country in fix_lst:
    df_EU_diesel[country] = df_EU_diesel[country].interpolate(method='slinear').interpolate(method='linear')

#Convert object to numeric
cols = df_EU_diesel.columns
df_EU_diesel[cols[1:]] = df_EU_diesel[cols[1:]].apply(pd.to_numeric, errors='coerce')    

#Calculate EU Average
df_EU_diesel.set_index('Date', inplace=True)
df_EU_diesel['Average'] = df_EU_diesel.mean(axis=1)
df_EU_diesel.reset_index(inplace=True)

df_EU_diesel['Year'] = df_EU_diesel['Date'].dt.year
df_EU_diesel['Month'] = df_EU_diesel['Date'].dt.month
df_EU_diesel = df_EU_diesel[['Year', 'Month', 'Average']]
df_EU_diesel.rename(columns = {'Average': 'EU_diesel'}, inplace = True)
display(df_EU_diesel)

C:\Users\Kydo\AppData\Local\Temp\ipykernel_7436\3051157815.py:16: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_EU_diesel[country] = df_EU_diesel[country].interpolate(method='slinear').interpolate(method='linear')


,Year,Month,EU_diesel
0,2015,1,1.339630
1,2015,2,1.329259
2,2015,3,1.323704
3,2015,4,1.321852
4,2015,5,1.394074
...,...,...,...
103,2023,8,1.804815
104,2023,9,1.836296
105,2023,10,1.824444
106,2023,11,1.803704


In [15]:
df_NA_diesel = df_NA[df_NA['PRODUCT'] == 'Diesel (unit/litre)']
df_NA_diesel.drop(columns = ['PRODUCT', 'FLOW', 'UNIT'], inplace = True)
#Dec 2020
#print(list(df_NA_diesel.columns))


df_NA_diesel.set_index('COUNTRY', inplace=True)
df_NA_diesel = df_NA_diesel.transpose()
df_NA_diesel.reset_index(inplace = True)
df_NA_diesel = df_NA_diesel.rename_axis(None, axis=1)
df_NA_diesel.rename(columns = {'index': 'Date'}, inplace = True)

#Convert object to numeric
cols = df_NA_diesel.columns
df_NA_diesel[cols[1:]] = df_NA_diesel[cols[1:]].apply(pd.to_numeric, errors='coerce')    

#Calculate EU Average
df_NA_diesel.set_index('Date', inplace=True)
df_NA_diesel['Average'] = df_NA_diesel.mean(axis=1)
df_NA_diesel.reset_index(inplace=True)

df_NA_diesel['Year'] = df_NA_diesel['Date'].dt.year
df_NA_diesel['Month'] = df_NA_diesel['Date'].dt.month
df_NA_diesel = df_NA_diesel[['Year', 'Month', 'Average']]
df_NA_diesel.rename(columns = {'Average': 'NA_diesel'}, inplace = True)
display(df_NA_diesel)

,Year,Month,NA_diesel
0,2015,1,0.855
1,2015,2,0.830
2,2015,3,0.860
3,2015,4,0.815
4,2015,5,0.845
...,...,...,...
103,2023,8,1.265
104,2023,9,1.315
105,2023,10,1.280
106,2023,11,1.235


### EU Supply from Eurostat

In [16]:
df_supply = pd.read_excel('EU_Supply.xlsx')
df_supply = df_supply.interpolate(method='linear')
df_supply = df_supply[['Year', 'Month', 'EU Gas & Diesel Supply', 'EU Gasoline Supply', 'EU Jet Fuel Supply']]
display(df_supply)

C:\Users\Kydo\AppData\Local\Temp\ipykernel_7436\606809269.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_supply = df_supply.interpolate(method='linear')


,Year,Month,EU Gas & Diesel Supply,EU Gasoline Supply,EU Jet Fuel Supply
0,2015,1,19810.000,5021.000,2700.000
1,2015,2,19569.000,4765.000,2499.000
2,2015,3,20606.000,5503.000,2863.000
3,2015,4,19725.000,5724.000,3103.000
4,2015,5,18278.000,5719.000,3371.000
...,...,...,...,...,...
103,2023,8,19174.067,6963.301,4354.859
104,2023,9,19174.067,6963.301,4354.859
105,2023,10,19174.067,6963.301,4354.859
106,2023,11,19174.067,6963.301,4354.859


### Air Activity (Passenger and Cargo) Extracted Manually from Monthly Reports of the IATA

In [17]:
df_air = pd.read_excel('Air_Demand.xlsx')
df_air = df_air.interpolate(method='linear')
df_air['Year'] = df_air['Date'].dt.year
df_air['Month'] = df_air['Date'].dt.month
df_air.drop(columns = ['Date'], inplace = True)
display(df_air)

,Africa_RPK_Billion,AsiaPacific_RPK_Billion,Europe_RPK_Billion,Latin America_RPK_Billion,MiddleEast_RPK_Billion,NorthAmerica_RPK_Billion,Africa_CTK_Billion,AsiaPacific_CTK_Billion,Europe_CTK_Billion,Latin America_CTK_Billion,MiddleEast_CTK_Billion,NorthAmerica_CTK_Billion,Year,Month
0,12.600000,154.875000,173.250000,26.250000,83.475000,70.350000,0.304200,6.979700,4.157400,0.490100,2.551900,2.433600,2015,1
1,12.537000,157.972500,174.289500,26.407500,83.725425,70.772100,0.306634,7.098355,4.219761,0.472456,2.659080,2.543112,2015,2
2,12.599685,156.392775,174.812368,26.433907,84.562679,70.772100,0.299274,6.842814,4.126926,0.471984,2.600580,2.449017,2015,3
3,12.511487,159.833416,175.511618,26.671813,84.562679,70.418240,0.297778,6.808600,4.118672,0.474344,2.629186,2.426976,2015,4
4,12.498976,161.431750,176.389176,27.018546,86.423058,70.770331,0.297182,6.699662,4.085723,0.467703,2.708062,2.402706,2015,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,12.369687,200.211424,186.344204,32.511954,95.421787,84.045714,0.493007,6.577494,4.259048,0.497573,2.940253,3.093335,2023,8
104,11.818380,189.736083,190.213890,32.261561,70.299516,63.735163,0.506035,6.846689,4.299252,0.479590,2.827683,2.875506,2023,9
105,13.459334,191.981162,207.119287,35.739274,117.921947,90.341675,0.510593,6.683252,4.270867,0.482776,2.949662,2.819058,2023,10
106,15.726751,204.532168,218.463298,35.747391,115.498930,89.670355,0.458997,6.674009,4.421189,0.471012,2.969827,2.791208,2023,11


### Google Search Trend - Manually Extracted from Google

In [18]:
df_ggst = pd.read_csv('Google_SearchTrend.csv')
df_ggst['Month'] = pd.to_datetime(df_ggst['Month'], errors='coerce')

df_ggst['Year'] = df_ggst['Month'].dt.year
df_ggst['Month'] = df_ggst['Month'].dt.month

df_ggst = df_ggst[(df_ggst['Year'] <= 2023) & (df_ggst['Year'] >= 2015)]

df_ggst.rename(columns = {'Search index': 'Google_SearchTrend'}, inplace = True)
display(df_ggst)

,Month,Google_SearchTrend,Year
0,1,57,2015
1,2,73,2015
2,3,85,2015
3,4,94,2015
4,5,94,2015
...,...,...,...
103,8,59,2023
104,9,69,2023
105,10,63,2023
106,11,60,2023


# Target Variable, end of processing

In [19]:
#Henkel
df_jetfuelprice = pd.read_excel('Henkel_JetFuel.xlsx', header = 4)
df_jetfuelprice = df_jetfuelprice[['Unnamed: 1', 'Jet Fuel Europe (US$/ton)', 'Jet Fuel US ($c/gallon)',
                                   'Jet Fuel US US$/MT']]
df_jetfuelprice.rename(columns = {'Unnamed: 1': 'Date'}, inplace = True)
df_jetfuelprice = df_jetfuelprice.iloc[1:217]
df_jetfuelprice['Date'] = pd.to_datetime(df_jetfuelprice['Date'], format='%Y-%m-%d %H:%M:%S')
df_jetfuelprice['Year'] = df_jetfuelprice['Date'].dt.year
df_jetfuelprice['Month'] = df_jetfuelprice['Date'].dt.month
df_jetfuelprice = df_jetfuelprice[(df_jetfuelprice['Year'] <= 2023) & (df_jetfuelprice['Year'] >= 2015)]
df_jetfuelprice = df_jetfuelprice[['Year', 'Month', 'Jet Fuel Europe (US$/ton)', 'Jet Fuel US ($c/gallon)',
                                   'Jet Fuel US US$/MT']]
df_jetfuelprice.rename(columns = {'Jet Fuel Europe (US$/ton)': 'JetFuelPrice_Europe'}, inplace = True)

#print(df_jetfuelprice.dtypes)
display(df_jetfuelprice)

,Year,Month,JetFuelPrice_Europe,Jet Fuel US ($c/gallon),Jet Fuel US US$/MT
109,2015,1,539.680000,148.64000,490.512000
110,2015,2,612.525000,177.71250,586.451250
111,2015,3,580.250000,162.86000,537.438000
112,2015,4,582.300000,169.03000,557.799000
113,2015,5,621.460000,184.16250,607.736250
...,...,...,...,...,...
212,2023,8,974.740000,298.82000,986.106000
213,2023,9,1037.520000,313.00625,1032.920625
214,2023,10,973.625000,288.90000,953.370000
215,2023,11,957.111457,284.00000,937.200000


# Merging

In [20]:
df = pd.merge(df_crudeoilprice, df_SP500)
df = pd.merge(df, df_STOXX600)
df = pd.merge(df, df_jetfuelprice)

df = pd.merge(df, df_EU_gas)
df = pd.merge(df, df_EU_diesel)
df = pd.merge(df, df_NA_gas)
df = pd.merge(df, df_NA_diesel)

df = pd.merge(df, df_ggst)

df = pd.merge(df, df_supply)
df = pd.merge(df, df_air)

df = pd.merge(df, df_eurusd)
df = pd.merge(df, df_gold)
df = pd.merge(df, df_inflation)
df = pd.merge(df, df_GDP)
df = pd.merge(df, df_GDPchange)
display(df)

,Year,Month,CrudeOilPrice_EU_Brent,CrudeOilPrice_US_WTI,SP500_Price,STOXX600_Price,JetFuelPrice_Europe,Jet Fuel US ($c/gallon),Jet Fuel US US$/MT,EU_Gasoline,...,GDPChange_Major advanced economies (G7),GDPChange_Other advanced economies (Advanced economies excluding G7 and euro area),GDPChange_European Union,GDPChange_ASEAN-5,GDPChange_Emerging market and developing economies,GDPChange_Emerging and developing Asia,GDPChange_Emerging and developing Europe,GDPChange_Latin America and the Caribbean,GDPChange_Middle East and Central Asia,GDPChange_Sub-Saharan Africa
0,2015,1,47.5,47.3,1994.99,367.05,539.680000,148.64000,490.512000,1.417407,...,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180
1,2015,2,58.2,50.6,2104.50,392.21,612.525000,177.71250,586.451250,1.407407,...,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180
2,2015,3,56.4,47.8,2067.89,397.30,580.250000,162.86000,537.438000,1.418519,...,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180
3,2015,4,59.5,54.5,2085.51,395.79,582.300000,169.03000,557.799000,1.446667,...,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180
4,2015,5,64.0,59.3,2107.39,399.87,621.460000,184.16250,607.736250,1.529630,...,2.060,2.333,2.496,4.590,4.326,6.823,1.013,0.251,3.011,3.180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2023,8,86.2,81.4,4507.66,458.19,974.740000,298.82000,986.106000,1.870741,...,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346
104,2023,9,94.0,89.6,4288.05,450.22,1037.520000,313.00625,1032.920625,1.862963,...,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346
105,2023,10,91.1,85.6,4193.80,433.66,973.625000,288.90000,953.370000,1.795926,...,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346
106,2023,11,83.1,77.4,4567.78,461.61,957.111457,284.00000,937.200000,1.777037,...,1.467,1.821,0.712,4.176,3.968,5.167,2.393,2.324,2.039,3.346


### Final renaming
This is typically unnecessary, but some libraries/programming languages are sensitive to space in the name of the column. This cell simply renames those columns and can be ignored if unnecessary (like Python).

In [21]:
df.rename(columns = {'JetFuelPrice_Europe': 'JetFuelPrice_Europe_USton',
           'Jet Fuel US ($c/gallon)': 'JetFuelPrice_US_cgallon',
           'Jet Fuel US US$/MT': 'JetFuelPrice_US_USMT',
                    'EU Gas & Diesel Supply': 'EU_GasDieselSupply',
                    'EU Jet Fuel Supply': 'EU_JetFuelSupply',
                    'GDP_Major advanced economies (G7)': 'GDP_G7',
                    'GDPChange_Major advanced economies (G7)': 'GDPChange_G7',
                    'Inflation_Major advanced economies (G7)': 'Inflation_G7'}, inplace = True)
df.columns = df.columns.str.replace(' ', '') 
print(df.columns)

Index(['Year', 'Month', 'CrudeOilPrice_EU_Brent', 'CrudeOilPrice_US_WTI',
       'SP500_Price', 'STOXX600_Price', 'JetFuelPrice_Europe_USton',
       'JetFuelPrice_US_cgallon', 'JetFuelPrice_US_USMT', 'EU_Gasoline',
       'EU_diesel', 'NA_Gasoline', 'NA_diesel', 'Google_SearchTrend',
       'EU_GasDieselSupply', 'EUGasolineSupply', 'EU_JetFuelSupply',
       'Africa_RPK_Billion', 'AsiaPacific_RPK_Billion', 'Europe_RPK_Billion',
       'LatinAmerica_RPK_Billion', 'MiddleEast_RPK_Billion',
       'NorthAmerica_RPK_Billion', 'Africa_CTK_Billion',
       'AsiaPacific_CTK_Billion', 'Europe_CTK_Billion',
       'LatinAmerica_CTK_Billion', 'MiddleEast_CTK_Billion',
       'NorthAmerica_CTK_Billion', 'EUR_USD_Rate', 'Gold_USD', 'Gold_EUR',
       'Inflation_World', 'Inflation_Advancedeconomies', 'Inflation_Euroarea',
       'Inflation_G7',
       'Inflation_Otheradvancedeconomies(AdvancedeconomiesexcludingG7andeuroarea)',
       'Inflation_EuropeanUnion', 'Inflation_ASEAN-5',
       'Inflatio

In [22]:
df.to_excel('Processed_Data.xlsx', index = False)